In [1]:
import torch 
import numpy as np
import imgaug.augmenters as iaa
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torchvision.transforms.functional as TF
from PIL import Image
import time

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
def Sharpening(p,T):
  return p**(1/T) / (p**(1/T)).sum(axis=0, keepdims=True)

In [16]:
Sharpening(torch.tensor([0.8,0.1,0.1]) , T=1.4)

tensor([0.6883, 0.1559, 0.1559])

In [4]:
def MixUp(x1,p1,x2,p2,alpha):
  lamda = np.random.beta(alpha, alpha) 
  lamda_h = np.maximum(lamda, 1-lamda)
  x = lamda_h * x1 + (1 - lamda_h) * x2
  p = lamda_h * p1.to(device) + (1 - lamda_h) * p2.to(device)
  return x,p

In [5]:
def mixMatch(model, x, P, u1, u2,size=32, T=0.5, K=2, alpha=0.75):
  P=torch.nn.functional.one_hot(P)
  b=x.shape[0]
  u_new = torch.cat((u1,u2),dim=0)
  label_i = model(u_new.to(device))
  labels=torch.zeros(2*P.shape[0],P.shape[1])
  for i in range(b):
    l=(label_i[i]+label_i[i+b])/2
    S=Sharpening(l,T)
    labels[i]=(S)
    labels[i+b]=(S)
  W = torch.cat([x, u_new],dim=0)
  Q = torch.cat((P, labels),dim=0)
  indices = np.random.shuffle(np.arange(len(W)))
  W=W[indices]
  Q=Q[indices]
  X_, p = MixUp(x, P, W[0][:len(x)], Q[0][:len(P)], alpha)
  U_, q = MixUp(u_new, labels, W[0][len(x):], Q[0][len(P):], alpha)
  # print("Pass MixMatch")
  return X_, U_, p, q

In [6]:
def ssLoss( p, pred_x, q, pred_u, lamda_u=100):
  l_x= torch.nn.CrossEntropyLoss()(pred_x,torch.max(p, 1)[1]) 
  l_u= torch.nn.MSELoss()(pred_u,q)
  return l_x + lamda_u*l_u

In [7]:
 import math
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    def __init__(self, in_planes, out_planes, stride, dropRate=0.0, activate_before_residual=False):
        super(BasicBlock, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes, momentum=0.001)
        self.relu1 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv1 = nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_planes, momentum=0.001)
        self.relu2 = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.conv2 = nn.Conv2d(out_planes, out_planes, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.droprate = dropRate
        self.equalInOut = (in_planes == out_planes)
        self.convShortcut = (not self.equalInOut) and nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride,
                               padding=0, bias=False) or None
        self.activate_before_residual = activate_before_residual
    def forward(self, x):
        if not self.equalInOut and self.activate_before_residual == True:
            x = self.relu1(self.bn1(x))
        else:
            out = self.relu1(self.bn1(x))
        out = self.relu2(self.bn2(self.conv1(out if self.equalInOut else x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        out = self.conv2(out)
        return torch.add(x if self.equalInOut else self.convShortcut(x), out)

class NetworkBlock(nn.Module):
    def __init__(self, nb_layers, in_planes, out_planes, block, stride, dropRate=0.0, activate_before_residual=False):
        super(NetworkBlock, self).__init__()
        self.layer = self._make_layer(block, in_planes, out_planes, nb_layers, stride, dropRate, activate_before_residual)
    def _make_layer(self, block, in_planes, out_planes, nb_layers, stride, dropRate, activate_before_residual):
        layers = []
        for i in range(int(nb_layers)):
            layers.append(block(i == 0 and in_planes or out_planes, out_planes, i == 0 and stride or 1, dropRate, activate_before_residual))
        return nn.Sequential(*layers)
    def forward(self, x):
        return self.layer(x)

class WideResNet(nn.Module):
    def __init__(self, num_classes, depth=28, widen_factor=2, dropRate=0.0):
        super(WideResNet, self).__init__()
        nChannels = [16, 16*widen_factor, 32*widen_factor, 64*widen_factor]
        assert((depth - 4) % 6 == 0)
        n = (depth - 4) / 6
        block = BasicBlock
        # 1st conv before any network block
        self.conv1 = nn.Conv2d(3, nChannels[0], kernel_size=3, stride=1,
                               padding=1, bias=False)
        # 1st block
        self.block1 = NetworkBlock(n, nChannels[0], nChannels[1], block, 1, dropRate, activate_before_residual=True)
        # 2nd block
        self.block2 = NetworkBlock(n, nChannels[1], nChannels[2], block, 2, dropRate)
        # 3rd block
        self.block3 = NetworkBlock(n, nChannels[2], nChannels[3], block, 2, dropRate)
        # global average pooling and classifier
        self.bn1 = nn.BatchNorm2d(nChannels[3], momentum=0.001)
        self.relu = nn.LeakyReLU(negative_slope=0.1, inplace=True)
        self.fc = nn.Linear(nChannels[3], num_classes)
        self.nChannels = nChannels[3]

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                m.bias.data.zero_()

    def forward(self, x):
        out = self.conv1(x)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(-1, self.nChannels)
        return self.fc(out)

In [56]:
wide_resnet28_model=WideResNet(num_classes=10)

In [9]:
optimizer = torch.optim.Adam(wide_resnet28_model.parameters(), weight_decay=0.0004 ) 

In [55]:
def train(model ,d_labled, d_unlabled, optimizer,num_epochs):
  iters=64 #int(len(d_unlabled.sampler)/d_unlabled.batch_size)
  print('iters ',iters)
  model.train()
  model = model.to(device)
  losses=[]
  accs=[]
  for epoch in range(num_epochs):
    begin=time.time()
    epoch_loss=0
    for i in range(iters):
      (x,p)=next(iter(d_labled))
      ((u1,u2),_)=next(iter(d_unlabled))
      X, U, p, q = mixMatch(model,x,p,u1,u2)
      X_pred= model(X.to(device))
      U_pred = model(U.to(device))
      loss = ssLoss(p,X_pred,q, U_pred)
      loss.backward()
      optimizer.step()
      epoch_loss+=loss.item()
    if epoch%2==0:
        acc=test(model, d_val, epoch)
        accs.append(acc)

    
    print(f'epoch {epoch} loss {epoch_loss/d_unlabled.batch_size}, time {time.time()-begin}')
    losses.append(epoch_loss/d_unlabled.batch_size)
  return accs , losses

In [47]:
def test(model,test_loader,epoch):
  model = model.to(device)
  model.eval()
  correct=0
  for x ,y in test_loader:
    output=model(x.to(device))
    pred = output.argmax(dim=1, keepdim=True)
    correct += pred.cpu().eq(y.view_as(pred)).sum().item()

  percent = 100. * correct / len(test_loader.sampler)
  print(f'epoch {epoch} Test accuracy: ({percent:.2f}%)')
  model.train()
  torch.save(model.state_dict(), f'{epoch}model.ckpt')
  return percent


In [12]:
trainset = torchvision.datasets.CIFAR10(root='./data',
                                        download=True)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [49]:
X_train =trainset.data 
y_train =trainset.targets

# # Train set / Validation set split 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1,
                                                          shuffle=True, stratify=y_train)

# Train unlabeled / Train labeled split
X_unlabled, X_labled, y_unlabeled, y_labeled = train_test_split(X_train, y_train, test_size=0.008, random_state=1,
                                                          shuffle=True, stratify=y_train)

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
normalize = transforms.Normalize(mean=mean, std=std)

t =  transforms.Compose([
                                         transforms.RandomRotation(30),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                         normalize])
t_val =  transforms.Compose([
                                        transforms.ToTensor(),
                                         normalize])

In [50]:
class two_transforms():
  def __init__(self):
    self.t1 = transforms.Compose([
                                         transforms.RandomRotation(30),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                         normalize])
    self.t2 = transforms.Compose([
                                         transforms.RandomRotation(30),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.ToTensor(),
                                         normalize])
  def __call__(self , x):
    return self.t1(x) , self.t2(x)
   

In [51]:
class MyData(Dataset):
    def __init__(self, x_train, y_train , transform):
        self.x_train = x_train
        self.y_train = y_train 
        self.transform = transform
        
    def __getitem__(self, index):
        image = self.transform(Image.fromarray(self.x_train[index]))
        label = self.y_train[index]
      
        return image, label
   
    def __len__(self):
        return self.x_train.shape[0]



In [52]:
d_labled = DataLoader( MyData(X_labled[:256], y_labeled[:256], t) ,batch_size=64,  shuffle=True,num_workers=4) #one tran
d_unlabled = DataLoader( MyData(X_unlabled, y_unlabeled, two_transforms()) , batch_size=64,shuffle=True,num_workers=4) #two tran
d_val = DataLoader( MyData(X_val, y_val, t_val) ,batch_size=64,  shuffle=False,num_workers=4) #one tran

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
accs , losses = train(wide_resnet28_model,d_labled, d_unlabled,optimizer,20)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(losses )